<a href="https://colab.research.google.com/github/AfricArxiv/decolonise-science/blob/master/AfricArxiv_stats_on_Zenodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Decolonise Science's Metadata extraction](https://github.com/AfricArxiv/decolonise-science/wiki/Metadata-extraction)

This was adapted from `zenodo-community-stats.sh` and configured to work on colab

Run the cells below to the end to get your extracted CSV file. You can also do `Runtime` -> `Run all` to run all automatically. 

Colab allows you to work with scripts adapted for other shells e.g jq used to parse json files is used in the ubuntu shell

sudo apt is used here to install the packages curl and jq. # To use this script you need to have "curl" and "jq" installed.

In [ ]:
! sudo apt-get install curl jq

We are using Python to set the variables because they do not get recognized just using `!`.

If you want to change the name of your extracted file, change the vairable `FILENAME`.

In [ ]:
import os
from datetime import datetime

# current date and time
curDT = datetime.now()
# current date
date = curDT.strftime("%Y-%m-%d")
# creating the date object of today's date
os.environ["date"] =  date
# Change below for your filename
FILENAME = f"decolsci_zenodo-extract_{date}"
os.environ["filename"] = FILENAME

In [ ]:
# Download all records (including multiple versions) from the community (max 10k records)
!curl "https://zenodo.org/api/records/?page=1&size=10000&q=&all_versions=true&communities=africarxiv" > zenodo-resp.json

In [ ]:
# Create CSV file header
!echo "URL,DOI,Title,Abstract,Views,Downloads,Author,Affiliation,ORCID,Created,Updated,License,Keywords" > "${filename}.csv"
# Process response and append to CSV file

In [ ]:
# Take first author information, parse timestamps, join keywords into a single string if array
!jq -r '.hits.hits[] | [.links.self, .metadata.doi, .metadata.title, .metadata.description, .stats.views, .stats.downloads, .metadata.creators[0].name, .metadata.creators[0].affiliation, .metadata.creators[0].orcid, (.created | split(".")[0] + "Z" | fromdate | strftime("%Y-%m-%d")), (.updated | split(".")[0] + "Z" | fromdate | strftime("%Y-%m-%d")), .metadata.license.id, (.metadata.keywords | if type == "array" then join(", ") else . end)] | @csv' zenodo-resp.json >> "${filename}.csv"

# Add Excel UTF-8 BOM to CSV file
!echo -ne "\xEF\xBB\xBF" | cat - "${filename}.csv" > temp.csv && mv temp.csv "${filename}.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  864k  100  864k    0     0   187k      0  0:00:04  0:00:04 --:--:--  211k
